In [1]:
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [2]:
def color_map(N=256, normalized=False):
    def bitget(byteval, idx):
        return ((byteval & (1 << idx)) != 0)

    dtype = 'float32' if normalized else 'uint8'
    cmap = np.zeros((N, 3), dtype=dtype)
    for i in range(N):
        r = g = b = 0
        c = i
        for j in range(8):
            r = r | (bitget(c, 0) << 7-j)
            g = g | (bitget(c, 1) << 7-j)
            b = b | (bitget(c, 2) << 7-j)
            c = c >> 3

        cmap[i] = np.array([r, g, b])

    cmap = cmap/255 if normalized else cmap
    return cmap


In [3]:
def labelToCode():
    labels = ['background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor', 'void']
    nclasses = 21
    d={}
    cmap = color_map()
    for i in range(nclasses):
        d[tuple(cmap[i].tolist())]=[labels[i],i+1]
    d[tuple(cmap[-1].tolist())]=[labels[-1],nclasses+1]
    return d


In [4]:
def resizeImage(image):
    resizedImage=np.zeros((224,224,3),dtype=np.int32)
    if(image.shape[0]>224):
        xDiff=image.shape[0]-224
        if(image.shape[1]>224):
            yDiff=image.shape[1]-224
            resizedImage[:,:,:]=image[(xDiff>>1):(xDiff>>1)+224,(yDiff>>1):(yDiff>>1)+224,:]
        else:
            yDiff=224-image.shape[1]
            resizedImage[:,(yDiff>>1):(yDiff>>1)+image.shape[1],:]=image[(xDiff>>1):(xDiff>>1)+224,:,:]
    else:
        xDiff=224-image.shape[0]
        if(image.shape[1]>224):
            yDiff=image.shape[1]-224
            resizedImage[(xDiff>>1):(xDiff>>1)+image.shape[0],:,:]=image[:,(yDiff>>1):(yDiff>>1)+224,:]
        else:
            yDiff=224-image.shape[1]
            resizedImage[(xDiff>>1):(xDiff>>1)+image.shape[0],(yDiff>>1):(yDiff>>1)+image.shape[1],:]=image[:,:,:]
    return resizedImage

In [5]:
def imageToMatrix(path,labels):
    image=cv2.imread(path)
    image=resizeImage(image)
    a=np.zeros((image.shape[0],image.shape[1],23),dtype=np.int32)
    for i in labels:
        temp=np.where(np.all(image == i[::-1], axis=-1))
        ind=(temp[0],temp[1],np.array(labels[i][-1]))
        a[ind]=1
    return a

In [6]:
imageToMatrix("E:\\DIP\\VOCtrain\\VOCdevkit\\VOC2012\\SegmentationClass\\2007_004468.png",labelToCode())

array([[[0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0]],

       [[0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0]],

       [[0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0]],

       ...,

       [[0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0]],

       [[0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 1, 0, ..., 

In [7]:
def discriminator(Images=None,reuse=False,sess=None):
    #Discriminator
    Parameters=[]
    with tf.variable_scope(tf.contrib.framework.get_name_scope(), reuse=reuse):
        #Placeholder
        if Images==None:
            Images=tf.placeholder(tf.float32,shape=(None,224,224,3),name='Dinput')
            vggmean=tf.constant([123.68,116.779,103.939], dtype=tf.float32, shape=[1,1,1,3])
            Images=(Images-vggmean)
        #Conv1
        W1=tf.get_variable("DW1",[3,3,3,64],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b1=tf.get_variable("Db1",[64],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv1=tf.nn.conv2d(Images,W1,[1,1,1,1],padding='SAME',name='Dconv1')
        z1=tf.nn.bias_add(conv1,b1,name='Dz1')
        a1=tf.nn.relu(z1,name='Da1')
        Parameters+=[W1,b1]
        #Conv2
        W2=tf.get_variable("DW2",[3,3,64,64],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b2=tf.get_variable("Db2",[64],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv2=tf.nn.conv2d(a1,W2,[1,1,1,1],padding='SAME',name='Dconv2')
        z2=tf.nn.bias_add(conv2,b2,name='Dz2')
        a2=tf.nn.relu(z2,name='Da2')
        Parameters+=[W2,b2]
        #Pool1
        pool1=tf.nn.max_pool(a2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name='Dpool1')
        #Conv3
        W3=tf.get_variable("DW3",[3,3,64,128],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b3=tf.get_variable("Db3",[128],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv3=tf.nn.conv2d(pool1,W3,[1,1,1,1],padding='SAME',name='Dconv3')
        z3=tf.nn.bias_add(conv3,b3,name='Dz3')
        a3=tf.nn.relu(z3,name='Da3')
        Parameters+=[W3,b3]
        #Conv4
        W4=tf.get_variable("DW4",[3,3,128,128],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b4=tf.get_variable("Db4",[128],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv4=tf.nn.conv2d(a3,W4,[1,1,1,1],padding='SAME',name='Dconv4')
        z4=tf.nn.bias_add(conv4,b4,name='Dz4')
        a4=tf.nn.relu(z4,name='Da4')
        Parameters+=[W4,b4]
        #Pool2
        pool2=tf.nn.max_pool(a4,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name='Dpool2')
        #Conv5
        W5=tf.get_variable("DW5",[3,3,128,256],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b5=tf.get_variable("Db5",[256],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv5=tf.nn.conv2d(pool2,W5,[1,1,1,1],padding='SAME',name='Dconv5')
        z5=tf.nn.bias_add(conv5,b5,name='Dz5')
        a5=tf.nn.relu(z5,name='Da5')
        Parameters+=[W5,b5]
        #Conv6
        W6=tf.get_variable("DW6",[3,3,256,256],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b6=tf.get_variable("Db6",[256],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv6=tf.nn.conv2d(a5,W6,[1,1,1,1],padding='SAME',name='Dconv6')
        z6=tf.nn.bias_add(conv6,b6,name='Dz6')
        a6=tf.nn.relu(z6,name='Da6')
        Parameters+=[W6,b6]
        #Conv7
        W7=tf.get_variable("DW7",[3,3,256,256],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b7=tf.get_variable("Db7",[256],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv7=tf.nn.conv2d(a6,W7,[1,1,1,1],padding='SAME',name='Dconv7')
        z7=tf.nn.bias_add(conv7,b7,name='Dz7')
        a7=tf.nn.relu(z7,name='Da7')
        Parameters+=[W7,b7]
        #Pool3
        pool3=tf.nn.max_pool(a7,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME",name='Dpool3')
        #Conv8
        W8=tf.get_variable("DW8",[3,3,256,512],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b8=tf.get_variable("Db8",[512],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv8=tf.nn.conv2d(pool3,W8,[1,1,1,1],padding='SAME',name='Dconv8')
        z8=tf.nn.bias_add(conv8,b8,name='Dz8')
        a8=tf.nn.relu(z8,name='Da8')
        Parameters+=[W8,b8]
        #Conv9
        W9=tf.get_variable("DW9",[3,3,512,512],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b9=tf.get_variable("Db9",[512],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv9=tf.nn.conv2d(a8,W9,[1,1,1,1],padding='SAME',name='Dconv9')
        z9=tf.nn.bias_add(conv9,b9,name='Dz9')
        a9=tf.nn.relu(z9,name='Da9')
        Parameters+=[W9,b9]
        #Conv10
        W10=tf.get_variable("DW10",[3,3,512,512],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b10=tf.get_variable("Db10",[512],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv10=tf.nn.conv2d(a9,W10,[1,1,1,1],padding='SAME',name='Dconv10')
        z10=tf.nn.bias_add(conv10,b10,name='Dz10')
        a10=tf.nn.relu(z10,name='Da10')
        Parameters+=[W10,b10]
        #Pool4
        pool4=tf.nn.max_pool(a10,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name='Dpool4')
        #Conv11
        W11=tf.get_variable("DW11",[3,3,512,512],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b11=tf.get_variable("Db11",[512],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv11=tf.nn.conv2d(pool4,W11,[1,1,1,1],padding='SAME',name='Dconv11')
        z11=tf.nn.bias_add(conv11,b11,name='Dz11')
        a11=tf.nn.relu(z11,name='Da11')
        Parameters+=[W11,b11]
        #Conv12
        W12=tf.get_variable("DW12",[3,3,512,512],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b12=tf.get_variable("Db12",[512],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv12=tf.nn.conv2d(a11,W12,[1,1,1,1],padding='SAME',name='Dconv12')
        z12=tf.nn.bias_add(conv12,b12,name='Dz12')
        a12=tf.nn.relu(z12,name='Da12')
        Parameters+=[W12,b12]
        #Conv13
        W13=tf.get_variable("DW13",[3,3,512,512],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32,trainable=False)
        b13=tf.get_variable("Db13",[512],initializer=tf.constant_initializer(0.0),dtype=tf.float32,trainable=False)
        conv13=tf.nn.conv2d(a12,W13,[1,1,1,1],padding='SAME',name='Dconv13')
        z13=tf.nn.bias_add(conv13,b13,name='Dz13')
        a13=tf.nn.relu(z13,name='Da13')
        Parameters+=[W13,b13]
        #Pool5
        pool5=tf.nn.max_pool(a13,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name='Dpool5')
        #Deconv1
        W14=tf.get_variable("DW14",[3,3,128,512],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
        b14=tf.get_variable("Db14",[128],initializer=tf.constant_initializer(0.0),dtype=tf.float32)
        outputshape1=[tf.shape(pool5)[0],28,28,128]
        deconv1=tf.nn.conv2d_transpose(pool5,W14,output_shape=outputshape1,strides=[1,4,4,1],padding='SAME',name='Dconv14')
        z14=tf.nn.bias_add(deconv1,b14,name='Dz14')
        a14=tf.nn.relu(z14,name='Da14')
        Parameters+=[W14,b14]
        #Deconv2
        W15=tf.get_variable("DW15",[3,3,32,128],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
        b15=tf.get_variable("Db15",[32],initializer=tf.constant_initializer(0.0),dtype=tf.float32)
        outputshape2=[tf.shape(a14)[0],112,112,32]
        deconv2=tf.nn.conv2d_transpose(a14,W15,output_shape=outputshape2,strides=[1,4,4,1],padding='SAME',name='Dconv15')
        z15=tf.nn.bias_add(deconv2,b15,name='Dz15')
        a15=tf.nn.relu(z15,name='Da15')
        Parameters+=[W15,b15]
        #Deconv3
        W16=tf.get_variable("DW16",[3,3,23,32],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
        b16=tf.get_variable("Db16",[23],initializer=tf.constant_initializer(0.0),dtype=tf.float32)
        outputshape3=[tf.shape(a15)[0],224,224,23]
        deconv3=tf.nn.conv2d_transpose(a15,W16,output_shape=outputshape3,strides=[1,2,2,1],padding='SAME',name='Dconv16')
        z16=tf.nn.bias_add(deconv3,b16,name='Dz16')
        Parameters+=[W16,b16]
        
    return z16

In [8]:
sess=tf.Session()

In [9]:
def generator(z=None,reuse=False):
    if not(z):
        z=tf.placeholder(tf.float32,shape=(None,100),name='GInput')
    with tf.variable_scope(tf.contrib.framework.get_name_scope(), reuse=reuse):
        #Linear
        GW1=tf.get_variable("GW1",[100,7*7*769],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
        Gb1=tf.get_variable("Gb1",[7*7*769],initializer=tf.constant_initializer(0.0),dtype=tf.float32)
        GprodTemp=tf.matmul(z,GW1,name="GprodTemp")
        Gprod=tf.nn.bias_add(GprodTemp,Gb1,name="Gprod")
        Gz1=tf.reshape(Gprod,[-1,7,7,769],name="Gz1")
        Ga1=tf.nn.relu(Gz1,name='Ga1')
        #Deconv1
        GW2=tf.get_variable("GW2",[3,3,384,769],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
        Gb2=tf.get_variable("Gb2",[384],initializer=tf.constant_initializer(0.0),dtype=tf.float32)
        Goutshape1=[tf.shape(Ga1)[0],14,14,384]
        Gdeconv1=tf.nn.conv2d_transpose(Ga1,GW2,output_shape=Goutshape1,strides=[1,2,2,1],padding="SAME",name="Gdeconv1")
        Gz2=tf.nn.bias_add(Gdeconv1,Gb2,name='Gz2')
        Ggamma1=tf.get_variable("Ggamma1",[1],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
        Gbeta1=tf.get_variable("Gbeta1",[1],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
        Gmean1,Gvar1=tf.nn.moments(Gz2,axes=[0],keep_dims=True)
        Gbatch1=tf.nn.batch_normalization(Gz2,Gmean1,Gvar1,Gbeta1,Ggamma1,1e-5,name='Gbatch1')
        Ga2=tf.nn.relu(Gbatch1,name='Ga2')
        #Deconv2
        GW3=tf.get_variable("GW3",[3,3,256,384],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
        Gb3=tf.get_variable("Gb3",[256],initializer=tf.constant_initializer(0.0),dtype=tf.float32)
        Goutshape2=[tf.shape(Ga2)[0],28,28,256]
        Gdeconv2=tf.nn.conv2d_transpose(Ga2,GW3,output_shape=Goutshape2,strides=[1,2,2,1],padding="SAME",name="Gdeconv2")
        Gz3=tf.nn.bias_add(Gdeconv2,Gb3,name='Gz3')
        Ggamma2=tf.get_variable("Ggamma2",[1],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
        Gbeta2=tf.get_variable("Gbeta2",[1],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
        Gmean2,Gvar2=tf.nn.moments(Gz3,axes=[0],keep_dims=True)
        Gbatch2=tf.nn.batch_normalization(Gz3,Gmean2,Gvar2,Gbeta2,Ggamma2,1e-5,name='Gbatch2')
        Ga3=tf.nn.relu(Gbatch2,name='Ga3')
        #Deconv3
        GW4=tf.get_variable("GW4",[3,3,192,256],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
        Gb4=tf.get_variable("Gb4",[192],initializer=tf.constant_initializer(0.0),dtype=tf.float32)
        Goutshape3=[tf.shape(Ga3)[0],112,112,192]
        Gdeconv3=tf.nn.conv2d_transpose(Ga3,GW4,output_shape=Goutshape3,strides=[1,4,4,1],padding="SAME",name="Gdeconv3")
        Gz4=tf.nn.bias_add(Gdeconv3,Gb4,name='Gz4')
        Ggamma3=tf.get_variable("Ggamma3",[1],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
        Gbeta3=tf.get_variable("Gbeta3",[1],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
        Gmean3,Gvar3=tf.nn.moments(Gz4,axes=[0],keep_dims=True)
        Gbatch3=tf.nn.batch_normalization(Gz4,Gmean3,Gvar3,Gbeta3,Ggamma3,1e-5,name='Gbatch3')
        Ga4=tf.nn.relu(Gbatch3,name='Ga4')
        #Deconv4
        GW5=tf.get_variable("GW5",[3,3,3,192],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
        Gb5=tf.get_variable("Gb5",[3],initializer=tf.constant_initializer(0.0),dtype=tf.float32)
        Goutshape4=[tf.shape(Ga4)[0],224,224,3]
        Gdeconv4=tf.nn.conv2d_transpose(Ga4,GW5,output_shape=Goutshape4,strides=[1,2,2,1],padding="SAME",name="Gdeconv4")
        Gz5=tf.nn.bias_add(Gdeconv4,Gb5,name='Gz5')
        Ggamma4=tf.get_variable("Ggamma4",[1],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
        Gbeta4=tf.get_variable("Gbeta4",[1],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
        Gmean4,Gvar4=tf.nn.moments(Gz5,axes=[0],keep_dims=True)
        Gbatch4=tf.nn.batch_normalization(Gz5,Gmean4,Gvar4,Gbeta4,Ggamma4,1e-5,name='Gbatch4')
        Ga5=tf.nn.tanh(Gbatch4,name='Ga5')
    return Ga5

sess.run(tf.global_variables_initializer())
print(sess.run(tf.report_uninitialized_variables()))

tf.trainable_variables()

out=tf.summary.FileWriter('./logs',sess.graph)
sess.close()

In [10]:
G=generator()

In [11]:
D1=discriminator(sess=sess)
D2=discriminator(reuse=True,sess=sess)
D3=discriminator(Images=G,reuse=True,sess=sess)

In [12]:
y=tf.placeholder(tf.float32,shape=(None,224,224,23),name='Output')

In [13]:
dloss_1=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=D1))

In [14]:
dloss_2=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(D2[:,:,:,0]),logits=D2[:,:,:,0]))

In [15]:
dloss_3=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(D3[:,:,:,0]),logits=D3[:,:,:,0]))

In [16]:
dloss=dloss_1+dloss_2+dloss_3

In [17]:
gloss=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(D3[:,:,:,0]),logits=D3[:,:,:,0]))

In [18]:
dloss,gloss

(<tf.Tensor 'add_1:0' shape=() dtype=float32>,
 <tf.Tensor 'Mean_3:0' shape=() dtype=float32>)

In [19]:
LabelledInput = tf.get_default_graph().get_tensor_by_name('Dinput:0')
UnlabelledInput = tf.get_default_graph().get_tensor_by_name('Dinput_1:0')
GeneratorInput = tf.get_default_graph().get_tensor_by_name('GInput:0')

In [20]:
LabelledImages=[]
for i in os.listdir("E:\\DIP\\VOCtrain\\VOCdevkit\\VOC2012\\SegmentationClass\\"):
    LabelledImages.append(i[:-3])
UnlabelledImages=[]
for i in os.listdir("E:\\DIP\\VOCtrain\\VOCdevkit\\VOC2012\\JPEGImages\\"):
    UnlabelledImages.append(i[:-3])
UnlabelledImages=list(set(UnlabelledImages)-set(LabelledImages))
UnlabelledImages=[i for i in UnlabelledImages]
LabelledImages=[i for i in LabelledImages] 

In [21]:
Dlist=[i for i in tf.get_default_graph().get_collection_ref(tf.GraphKeys.TRAINABLE_VARIABLES) if i.name[0]=='D']

In [22]:
Glist=[i for i in tf.get_default_graph().get_collection_ref(tf.GraphKeys.TRAINABLE_VARIABLES) if i.name[0]=='G']

In [23]:
Dlist,Glist

([<tf.Variable 'DW14:0' shape=(3, 3, 128, 512) dtype=float32_ref>,
  <tf.Variable 'Db14:0' shape=(128,) dtype=float32_ref>,
  <tf.Variable 'DW15:0' shape=(3, 3, 32, 128) dtype=float32_ref>,
  <tf.Variable 'Db15:0' shape=(32,) dtype=float32_ref>,
  <tf.Variable 'DW16:0' shape=(3, 3, 23, 32) dtype=float32_ref>,
  <tf.Variable 'Db16:0' shape=(23,) dtype=float32_ref>],
 [<tf.Variable 'GW1:0' shape=(100, 37681) dtype=float32_ref>,
  <tf.Variable 'Gb1:0' shape=(37681,) dtype=float32_ref>,
  <tf.Variable 'GW2:0' shape=(3, 3, 384, 769) dtype=float32_ref>,
  <tf.Variable 'Gb2:0' shape=(384,) dtype=float32_ref>,
  <tf.Variable 'Ggamma1:0' shape=(1,) dtype=float32_ref>,
  <tf.Variable 'Gbeta1:0' shape=(1,) dtype=float32_ref>,
  <tf.Variable 'GW3:0' shape=(3, 3, 256, 384) dtype=float32_ref>,
  <tf.Variable 'Gb3:0' shape=(256,) dtype=float32_ref>,
  <tf.Variable 'Ggamma2:0' shape=(1,) dtype=float32_ref>,
  <tf.Variable 'Gbeta2:0' shape=(1,) dtype=float32_ref>,
  <tf.Variable 'GW4:0' shape=(3, 3, 19

In [24]:
sess.run(tf.global_variables_initializer())
weights=np.load("E:\\DIP\\vgg16_weights.npz")
j=0
Parameters=[i for i in tf.get_default_graph().get_collection_ref(tf.GraphKeys.GLOBAL_VARIABLES) if i.name[0]=='D']
for i in sorted(weights.keys()):
    if j>=26:
        break
    else:
        sess.run(Parameters[j].assign(weights[i]))
    j+=1

In [33]:
DiscTrain=tf.train.AdamOptimizer(0.00001).minimize(dloss,var_list=Dlist)
GenTrain=tf.train.AdamOptimizer(0.000000002).minimize(gloss,var_list=Glist)
sess.run(tf.global_variables_initializer())

In [34]:
numberofepochs=1

In [37]:
r=labelToCode()
for i in range(numberofepochs):
    print("EPOCH",i)
    for j in range(0,len(LabelledImages)-1,2):
        BatchInput=[]
        BatchInput.append(resizeImage(cv2.imread("E:\\DIP\\VOCtrain\\VOCdevkit\\VOC2012\\JPEGImages\\"+LabelledImages[j]+'jpg')[:,:,::-1]))
        BatchInput.append(resizeImage(cv2.imread("E:\\DIP\\VOCtrain\\VOCdevkit\\VOC2012\\JPEGImages\\"+LabelledImages[j+1]+'jpg')[:,:,::-1]))
        BatchInput=np.array(BatchInput,dtype=np.float32)
        Output=[]
        Output.append(imageToMatrix("E:\\DIP\\VOCtrain\\VOCdevkit\\VOC2012\\SegmentationClass\\"+LabelledImages[j]+'png',r))
        Output.append(imageToMatrix("E:\\DIP\\VOCtrain\\VOCdevkit\\VOC2012\\SegmentationClass\\"+LabelledImages[j+1]+'png',r))        
        Output=np.array(Output,dtype=np.float32)
        UnlabelledInput=[]
        UnlabelledInput.append(resizeImage(cv2.imread("E:\\DIP\\VOCtrain\\VOCdevkit\\VOC2012\\JPEGImages\\"+UnlabelledImages[np.random.randint(0,14000)]+'jpg')[:,:,::-1]))
        UnlabelledInput.append(resizeImage(cv2.imread("E:\\DIP\\VOCtrain\\VOCdevkit\\VOC2012\\JPEGImages\\"+UnlabelledImages[np.random.randint(0,14000)]+'jpg')[:,:,::-1]))
        UnlabelledInput=np.array(UnlabelledInput,dtype=np.float32)
        Noise=np.array([np.random.randn(100),np.random.randn(100)],dtype=np.float32)
        #print(Noise.dtype)
        val={'Dinput:0':BatchInput,'Dinput_1:0':UnlabelledInput,'GInput:0':Noise,y:Output}
        #print(val)
        _,loss1=sess.run([DiscTrain,dloss],feed_dict=val)
        _,loss2=sess.run([GenTrain,gloss],feed_dict=val)
        print(i,j,loss1,loss2)
        #print(sess.run(dloss_3,feed_dict=val))

EPOCH 0
0 0 3.8737712 0.6802483
0 2 4.1280766 0.6802459
0 4 4.052579 0.68024415
0 6 3.9981136 0.68024224
0 8 4.249869 0.6802432
0 10 4.291905 0.6802453
0 12 3.9297554 0.6802453
0 14 4.065435 0.6802432
0 16 4.056595 0.6802406
0 18 4.0479593 0.68023837
0 20 4.078666 0.68023634
0 22 4.034344 0.68023455
0 24 3.8949175 0.6802316
0 26 4.511838 0.68023354
0 28 3.6289518 0.6802312
0 30 3.7571034 0.680227
0 32 4.2608294 0.6802256
0 34 3.825182 0.6802231
0 36 3.9123306 0.6802214
0 38 4.1200986 0.6802198
0 40 4.0272274 0.68021804
0 42 4.0345573 0.68021715
0 44 3.6516707 0.68021286
0 46 4.1299405 0.6802091
0 48 4.029875 0.68020624
0 50 3.9866972 0.6802035
0 52 3.844334 0.6802
0 54 3.9478524 0.6801961
0 56 3.8179464 0.68018997
0 58 3.9212914 0.68018144
0 60 4.0712223 0.6801748
0 62 3.9485486 0.6801687
0 64 3.697466 0.6801611
0 66 3.929854 0.6801525
0 68 3.9088988 0.6801427
0 70 4.035878 0.6801339
0 72 4.000942 0.6801242
0 74 3.859721 0.6801157
0 76 4.042074 0.68010855
0 78 3.8889086 0.6801001
0 80 

0 636 3.7104447 0.679053
0 638 3.7773948 0.6790495
0 640 3.9454882 0.6790452
0 642 3.8783927 0.6790416
0 644 3.8161874 0.67903775
0 646 4.007009 0.6790347
0 648 3.7784925 0.67903006
0 650 4.268382 0.6790291
0 652 3.6148345 0.67902595
0 654 4.1058407 0.6790225
0 656 3.9155946 0.6790186
0 658 3.9494805 0.6790143
0 660 3.6501896 0.67900705
0 662 3.7808645 0.6789988
0 664 4.0286818 0.6789917
0 666 3.6740787 0.6789833
0 668 3.7424586 0.67897475
0 670 3.681675 0.6789659
0 672 3.9302177 0.6789585
0 674 3.774003 0.67894995
0 676 3.8966296 0.6789421
0 678 3.8277235 0.6789347
0 680 4.0332108 0.67892915
0 682 3.8351927 0.67892283
0 684 3.854228 0.6789182
0 686 3.720263 0.67891294
0 688 3.6784177 0.6789068
0 690 3.8202069 0.6789007
0 692 3.4866636 0.6788926
0 694 4.072811 0.67888623
0 696 3.4768095 0.67887557
0 698 4.0422764 0.6788655
0 700 3.8435497 0.6788547
0 702 3.6808147 0.67884284
0 704 4.170417 0.67883253
0 706 3.8170033 0.6788221
0 708 3.9885843 0.67881244
0 710 3.753615 0.6788035
0 712 3.

0 1260 4.045513 0.67873275
0 1262 4.163373 0.6787359
0 1264 4.061499 0.6787402
0 1266 3.81344 0.6787434
0 1268 4.133268 0.67874706
0 1270 3.7389941 0.67874986
0 1272 3.8331742 0.67875266
0 1274 3.440683 0.67875206
0 1276 3.8665423 0.67875135
0 1278 3.6719556 0.67874926
0 1280 3.9468148 0.67874676
0 1282 4.177034 0.6787455
0 1284 3.7327387 0.6787417
0 1286 3.949574 0.6787395
0 1288 4.0676227 0.67873925
0 1290 3.5792685 0.6787366
0 1292 3.6101882 0.67873055
0 1294 3.785386 0.6787239
0 1296 3.7541587 0.67871684
0 1298 4.0115085 0.6787119
0 1300 3.8556278 0.6787072
0 1302 3.7977543 0.6787013
0 1304 4.176685 0.6786977
0 1306 3.8140228 0.67869425
0 1308 3.9523172 0.6786919
0 1310 3.9251442 0.67869055
0 1312 4.194161 0.67869157
0 1314 3.8439581 0.6786918
0 1316 3.814199 0.67869085
0 1318 3.7571175 0.6786898
0 1320 3.6566253 0.6786868
0 1322 3.8357172 0.67868173
0 1324 3.4593856 0.6786762
0 1326 3.8381658 0.67867106
0 1328 3.579424 0.67866254
0 1330 4.255098 0.67865807
0 1332 3.9736726 0.67865

0 1870 3.8097415 0.6794391
0 1872 4.0908375 0.679441
0 1874 3.6844127 0.6794415
0 1876 3.6360743 0.67944086
0 1878 4.014613 0.67944103
0 1880 3.4430997 0.6794378
0 1882 4.0391817 0.6794367
0 1884 3.6979804 0.6794344
0 1886 4.1005244 0.6794341
0 1888 4.335329 0.67943853
0 1890 4.17613 0.6794451
0 1892 4.105522 0.6794527
0 1894 4.0491686 0.67946064
0 1896 3.406477 0.67946345
0 1898 3.9228299 0.67946607
0 1900 3.9632006 0.67946863
0 1902 3.878475 0.6794695
0 1904 3.9006462 0.67947066
0 1906 4.485142 0.67947716
0 1908 4.022742 0.67948383
0 1910 4.276152 0.6794929
0 1912 3.659292 0.67949784
0 1914 3.835742 0.6795019
0 1916 4.1998878 0.6795097
0 1918 3.850618 0.6795158
0 1920 3.633334 0.67951715
0 1922 4.071155 0.67951864
0 1924 3.654589 0.67951727
0 1926 4.145177 0.67951894
0 1928 3.9533825 0.6795201
0 1930 4.611045 0.679527
0 1932 4.2001133 0.67953634
0 1934 4.0447836 0.67954516
0 1936 4.114852 0.67955554
0 1938 3.5732374 0.679564
0 1940 3.5866761 0.67957073
0 1942 3.7966614 0.6795773
0 19

KeyboardInterrupt: 

In [36]:
sess.run(tf.get_default_graph().get_tensor_by_name('DW14:0'))

array([[[[-0.00549919,  0.03337722,  0.01730994, ...,  0.00427328,
           0.01704031,  0.00097677],
         [ 0.0005865 , -0.00700154,  0.00911332, ..., -0.00948222,
           0.00955431,  0.03342534],
         [-0.02283746,  0.0144736 ,  0.00388372, ..., -0.02760927,
          -0.00458399, -0.02616526],
         ...,
         [-0.00557281, -0.00057682,  0.03321423, ..., -0.00298839,
           0.03125343, -0.00105238],
         [-0.00326239,  0.04375004,  0.01176238, ..., -0.0105947 ,
           0.03197195, -0.01477838],
         [ 0.01482033,  0.03792546, -0.01172991, ..., -0.00399324,
           0.01354544,  0.00412283]],

        [[-0.01179501,  0.04332247,  0.00641119, ..., -0.00263163,
           0.01271267,  0.01368327],
         [ 0.01604692,  0.01819888, -0.02839476, ...,  0.01422643,
           0.01346793, -0.02895385],
         [-0.0081243 ,  0.01863332,  0.03358532, ...,  0.02587189,
           0.03678839,  0.0181729 ],
         ...,
         [ 0.00088675,  0.04629343

In [30]:
weights['conv5_3_W']

array([[[[ 4.07882966e-04,  4.82621184e-03, -9.11695044e-03, ...,
           1.51569908e-02, -2.73966836e-03, -4.83311387e-03],
         [ 3.48924054e-03,  2.60589737e-03, -8.10198858e-03, ...,
           8.86987546e-04, -7.08238874e-03, -7.96321197e-04],
         [-1.12759033e-02, -6.43836753e-03, -8.05236772e-03, ...,
           1.90410519e-03,  3.87627119e-03,  1.96338408e-02],
         ...,
         [-1.49949323e-02,  1.20318262e-02,  9.87236737e-04, ...,
          -2.89861043e-03,  4.03315574e-03, -5.93429338e-03],
         [-7.53336679e-03,  6.08530594e-03,  9.90014290e-04, ...,
           1.63243050e-04, -3.09621776e-03, -3.01808352e-03],
         [-2.62522907e-03,  3.51887429e-03, -2.58309790e-03, ...,
          -6.02027494e-03, -8.85035843e-03,  9.85825085e-04]],

        [[ 3.02846078e-04,  2.64736195e-03, -1.37099335e-02, ...,
           1.49135189e-02, -4.65666235e-04, -6.84898719e-03],
         [-9.60117579e-03,  5.01031429e-03, -1.13053191e-02, ...,
           8.86707660e

0 0 4.5219398 0.6931415
0 2 4.521087 0.6931358
0 4 4.521656 0.6931305
0 6 4.5212517 0.69312507
0 8 4.5208836 0.6931195
0 10 4.5215898 0.6931138